In [21]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
import os

import logging
logging.basicConfig(level=logging.ERROR)
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent
from pydantic import BaseModel, Field
from google.adk.agents.parallel_agent import ParallelAgent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts


In [22]:
from yt_transcript import *

yt_url = "https://www.youtube.com/watch?v=sBuoMkJsMsA"

In [23]:
AGENT_MODEL = "gemini-2.0-flash"
root_agent = Agent(
    name="yt_agent_v0",
    model=AGENT_MODEL, 
    description="Extracts transcripts from YouTube videos and summarizes the learnings related to a given topic.",
    instruction = (
    "You are an expert YouTube video summarizer. "
    "Given a topic and a YouTube video URL, use the 'get_youtube_transcript' tool to extract the transcript (returned as a JSON with key 'result'). "
    "From the transcript, generate a clear and concise summary that captures the key learnings relevant to the given topic. "
    "Your output should be only the final summarized content — answer in bullet points."
    "access the video url from session.state['q1']"
    ),
    output_key='yt_agent_v0', 
    tools=[get_youtube_transcript],
)

root_agent_1 = Agent(
    name="yt_agent_v1",
    model=AGENT_MODEL, 
    description="Extracts transcripts from YouTube videos and summarizes the learnings related to a given topic.",
    instruction = (
    "You are an expert YouTube video summarizer. "
    "Given a topic and a YouTube video URL, use the 'get_youtube_transcript' tool to extract the transcript (returned as a JSON with key 'result'). "
    "From the transcript, generate a clear and concise summary that captures the key learnings relevant to the given topic. "
    "Your output should be only the final summarized content — answer in bullet points."
    "access the video url from session.state['q2']"
    ),
    output_key='yt_agent_v1',
    tools=[get_youtube_transcript],
)

root_agent_2 = Agent(
    name="yt_agent_v2",
    model=AGENT_MODEL, 
    description="Extracts transcripts from YouTube videos and summarizes the learnings related to a given topic.",
    instruction = (
    "You are an expert YouTube video summarizer. "
    "Given a topic and a YouTube video URL, use the 'get_youtube_transcript' tool to extract the transcript (returned as a JSON with key 'result'). "
    "From the transcript, generate a clear and concise summary that captures the key learnings relevant to the given topic. "
    "Your output should be only the final summarized content — answer in bullet points."
    "access the video url from session.state['q3']"
    ),
    output_key='yt_agent_v2',
    tools=[get_youtube_transcript],
)

In [24]:
session_service = InMemorySessionService()

APP_NAME = "yt_app"
USER_ID = "user_1"
SESSION_ID = "session_001" 

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
runner = Runner(
    agent=root_agent, 
    app_name=APP_NAME,   
    session_service=session_service #
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='yt_app', User='user_1', Session='session_001'
Runner created for agent 'yt_agent_v0'.


In [25]:
import asyncio
from google.genai import types 

async def call_agent_async(query: str, runner: Runner):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." #default

  async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          break 

  print(f"<<< Agent Response: {final_response_text}")
  return final_response_text

In [26]:
text = await call_agent_async(f"how to swim? yt_url= {yt_url}", runner)


>>> User Query: how to swim? yt_url= https://www.youtube.com/watch?v=sBuoMkJsMsA
<<< Agent Response: Here's a summary of how to swim freestyle, based on the video transcript:

*   **Step 1: Streamline.** Practice gliding on the surface with your head down and arms stretched forward. Focus on pushing off the wall and maintaining a streamlined position.
*   **Step 2: Arm Strokes with a Board.** Use a kickboard to practice arm strokes, keeping your elbows high and pushing the water with your hands and forearms. Feel the pressure in the water as you pull. Then, practice the same movement without the board, gliding forward with your hands on the surface.
*   **Step 3: Freestyle with Breathing.** Incorporate breathing into your stroke. After every third stroke, turn your head to the side to breathe, ensuring your eye remains on the water and you're not lifting your head too high. Alternate breathing to the left and right sides.
*   **Real Freestyle:** Instead of touching hands together, gli

In [27]:
session = session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
session.state['yt_agent_v0']

"Here's a summary of how to swim freestyle, based on the video transcript:\n\n*   **Step 1: Streamline.** Practice gliding on the surface with your head down and arms stretched forward. Focus on pushing off the wall and maintaining a streamlined position.\n*   **Step 2: Arm Strokes with a Board.** Use a kickboard to practice arm strokes, keeping your elbows high and pushing the water with your hands and forearms. Feel the pressure in the water as you pull. Then, practice the same movement without the board, gliding forward with your hands on the surface.\n*   **Step 3: Freestyle with Breathing.** Incorporate breathing into your stroke. After every third stroke, turn your head to the side to breathe, ensuring your eye remains on the water and you're not lifting your head too high. Alternate breathing to the left and right sides.\n*   **Real Freestyle:** Instead of touching hands together, glide with hands one by one. Open your hands like your shoulders while you're here and avoid being 

# Parallel Agent

In [28]:
# parallel_research_agent = ParallelAgent(
#     name="ParallelWebResearchAgent",
#     sub_agents=[root_agent, root_agent_1, root_agent_2]
# )

In [29]:
# APP_NAME = "yt_app_advanced"
# USER_ID = "user_2"
# SESSION_ID = "session_002" 

# Session and Runner
# session_service = InMemorySessionService()
# session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
# runner = Runner(agent=parallel_research_agent, app_name=APP_NAME, session_service=session_service)


# # Agent Interaction
# def call_agent(query):
#     '''
#     Helper function to call the agent with a query.
#     '''
#     content = types.Content(role='user', parts=[types.Part(text=query)])
#     events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

#     for event in events:
#         if event.is_final_response():
#             final_response = event.content.parts[0].text
#             print("Agent Response: ", final_response)

# call_agent(f"how to swim? yt_url= {yt_url}")

In [30]:
yt_url_1 ="https://www.youtube.com/watch?v=sBuoMkJsMsA"
yt_url_2 = "https://www.youtube.com/watch?v=1nnndhSc4iw"
yt_url_3 = "https://www.youtube.com/watch?v=_kGESn8ArrU"

In [31]:
# async def process_multiple_queries(queries):    
#     # Create a trigger message to start processing
#     query = f"Process all queries in parallel in the sequence {queries['q1']}, {queries['q2']}, {queries['q3']}"
#     message = types.Content(
#         role="user", 
#         parts=[types.Part(text=query)]
#     )
#     session = session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
#     # Run the parallel agent
#     async for event in runner.run_async(
#         user_id=USER_ID, 
#         session_id=SESSION_ID, 
#         new_message=message
#     ):
#         if event.is_final_response():
#             # Processing complete, now retrieve results from session state
#             results = {
#                 "q1": session.state.get("yt_agent_v0"),
#                 "q2": session.state.get("yt_agent_v1"),
#                 "q3": session.state.get("yt_agent_v2")
#             }
#             return results


# queries = {
#     "q1": f"how to swim? yt_url= {yt_url_1}",
#     "q2": f"how to sing? yt_url= {yt_url_2}",
#     "q3": f"how to run? yt_url= {yt_url_3}"
# }
# results = await process_multiple_queries(queries)
# print(results)

In [32]:
# session = session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
# session.state

# Iterating multiple queries

In [33]:
queries = {
    "q1": f"how to swim? yt_url= {yt_url_1}",
    "q2": f"how to sing? yt_url= {yt_url_2}",
    "q3": f"how to run? yt_url= {yt_url_3}"
}

In [34]:
text = [await call_agent_async(f"{query}", runner) for query in queries.values()]


>>> User Query: how to swim? yt_url= https://www.youtube.com/watch?v=sBuoMkJsMsA
<<< Agent Response: *   **Step 1: Streamline:** Practice gliding on the surface with your head down and arms stretched forward.
*   **Step 2: Arm Strokes with a Board:** Use a kickboard, keep elbows high, and feel the pressure as you pull. Then, practice without the board, gliding with hands on the surface.
*   **Step 3: Freestyle with Breathing:** Breathe every third stroke, turning your head to the side, keeping your eye on the water.
*   **Real Freestyle:** Glide with alternating hands, opening them like shoulders, maintaining smooth motion.

>>> User Query: how to sing? yt_url= https://www.youtube.com/watch?v=1nnndhSc4iw
<<< Agent Response: Here's a summary of how to sing better instantly, based on the video transcript:

*   **Find Your Voice's Resting Place:** Start by sighing with your mouth closed (hmm) to find a strain-free vocal baseline. Then, do a glissando (sliding down through notes) to find 

In [35]:
main_agent = Agent(
    name="yt_agent_main",
    model=AGENT_MODEL, 
    description="Summurises the whole knowledge based on multiple youtube video summuries.",
    instruction = (
    "You are an expert YouTube video summarizer. "
    "Given a topic and multiple short summary from YouTube videos, you are asked to elaborate the whole knowledge. "
    "From the summary, generate a clear and precise summary that captures the key learnings relevant to the given topic. "
    "Your output should be only the final content — answer in bullet points."
    ),
    output_key='yt_agent_main',
)

In [36]:
import asyncio
from google.genai import types 

runner_main = Runner(
    agent=main_agent, 
    app_name=APP_NAME,   
    session_service=session_service 
)

In [37]:
await call_agent_async(f"{text}", runner_main)


>>> User Query: ['*   **Step 1: Streamline:** Practice gliding on the surface with your head down and arms stretched forward.\n*   **Step 2: Arm Strokes with a Board:** Use a kickboard, keep elbows high, and feel the pressure as you pull. Then, practice without the board, gliding with hands on the surface.\n*   **Step 3: Freestyle with Breathing:** Breathe every third stroke, turning your head to the side, keeping your eye on the water.\n*   **Real Freestyle:** Glide with alternating hands, opening them like shoulders, maintaining smooth motion.', "Here's a summary of how to sing better instantly, based on the video transcript:\n\n*   **Find Your Voice's Resting Place:** Start by sighing with your mouth closed (hmm) to find a strain-free vocal baseline. Then, do a glissando (sliding down through notes) to find the note where your voice rests most comfortably.\n*   **Drop Your Jaw:** Once you've found that comfortable note, drop your jaw while maintaining the same relaxed feeling as th

"Here's a comprehensive guide incorporating key learnings from swimming, singing, and running:\n\n*   **Swimming (Freestyle):**\n    *   **Streamline:** Begin by practicing a streamlined glide on the surface, keeping your head down and arms stretched forward to reduce drag.\n    *   **Arm Strokes:** Use a kickboard to focus on arm strokes, maintaining high elbows and feeling the water pressure as you pull. Practice the same motion without the board, gliding with your hands along the surface.\n    *   **Breathing:** Integrate breathing every third stroke by turning your head to the side, keeping one eye in the water to maintain balance and body position.\n    *   **Freestyle Technique:** Emphasize gliding with alternating hands, opening them like your shoulders to maintain a smooth, efficient motion through the water.\n\n*   **Singing:**\n    *   **Find Your Natural Voice:** Start by humming to find a relaxed, strain-free vocal baseline. Use a glissando (sliding between notes) to identi